In [1]:
#| default_exp 24_msmarco-hard-negatives-systems

In [2]:
%reload_ext autoreload
%autoreload 2

In [6]:
#| export
import pickle, scipy.sparse as sp, numpy as np, argparse, os, json
from tqdm.auto import tqdm
from typing import Optional, List

from xcai.main import *
from sugar.core import *

## Setup

In [7]:
fname = '/Users/suchith720/Projects/data/msmarco-hard-negatives.jsonl'

In [32]:
#| export
def load_negatives_system_info(fname):
    negatives, systems = {}, {}
    with open(fname) as file:
        for line in file:
            c = json.loads(line)
            for k,v in c['neg'].items():
                sys_id = systems.setdefault(k, len(systems))
                negatives[c['qid']] = {o:sys_id for o in v}
    return systems, negatives
                

In [29]:
%%time
systems, negatives = load_negatives_system_info(fname)

CPU times: user 1min 1s, sys: 1.53 s, total: 1min 2s
Wall time: 1min 2s


In [31]:
len(systems)

13